In [1]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow import keras
import numpy as np
from os import *
from os import listdir
from os.path import isfile
import math
from prepare_data import *

from datetime import datetime
%load_ext tensorboard

In [2]:
dataset = Dataset.carabid
extractor_train_gen, extractor_val_gen, extractor_test_gen = prep_data_single(dataset, 8)
train_gen, val_gen, test_gen = prep_data(dataset, 8, 3)

In [3]:
inception_model = InceptionV3(classifier_activation=None)
for layer in inception_model.layers:
    layer.trainable = True

In [4]:
extractor_logdir = "logs/time_distributed/{0}_{1}/extractor".format(str(dataset), datetime.now().strftime("%Y%m%d-%H%M%S"))
extractor_tensorboard_callback = keras.callbacks.TensorBoard(log_dir=extractor_logdir)

extractor_model_path = "models/time_distributed/{0}_{1}/extractor/savefile".format(str(dataset), datetime.now().strftime("%Y%m%d-%H%M%S"))
extractor_model_save_callback = keras.callbacks.ModelCheckpoint(filepath=extractor_model_path, save_weights_only=True, verbose=1)

In [6]:
feature_extractor = keras.Sequential([inception_model, keras.layers.Dense(train_gen.num_classes(), activation='softmax')])
feature_extractor.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
feature_extractor.fit(extractor_train_gen, validation_data=extractor_val_gen, callbacks=[extractor_tensorboard_callback, extractor_model_save_callback], epochs=1)

5588/5588 [==============================] - 526s 92ms/step - loss: 4.9683 - accuracy: 0.0313 - val_loss: 4.4644 - val_accuracy: 0.0800

Epoch 00001: saving model to models/time_distributed/Dataset.carabid_20220221-214353/extractor\savefile


In [12]:
print(feature_extractor.layers[0].layers[-5:])

[<tensorflow.python.keras.layers.merge.Concatenate object at 0x000001CE88C6C5E0>, <tensorflow.python.keras.layers.core.Activation object at 0x000001CE88C87DC0>, <tensorflow.python.keras.layers.merge.Concatenate object at 0x000001CE88C775B0>, <tensorflow.python.keras.layers.pooling.GlobalAveragePooling2D object at 0x000001CE88C6CD90>, <tensorflow.python.keras.layers.core.Dense object at 0x000001CEFC8AC2E0>]


In [14]:
inception_model = feature_extractor.layers[0]
for layer in inception_model.layers:
    layer.trainable = False

inputs = keras.layers.Input(shape=(None, 299, 299, 3,))
extractor_layer = keras.layers.TimeDistributed(inception_model)(inputs)
lstm_layer = keras.layers.LSTM(1000)(extractor_layer)
dropout_layer = keras.layers.Dropout(0.5)
dense_layer = keras.layers.Dense(1000, activation='relu')(dropout_layer)
predictions = keras.layers.Dense(train_gen.num_classes(), activation='softmax')(dense_layer)

# lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.1, decay_steps=1000, decay_rate=0.9)
model = keras.Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=keras.optimizers.Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None, 299, 299, 3 0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1000)        23851784  
_________________________________________________________________
lstm_1 (LSTM)                (None, 1000)              8004000   
_________________________________________________________________
dense_4 (Dense)              (None, 1000)              1001000   
_________________________________________________________________
dense_5 (Dense)              (None, 291)               291291    
Total params: 33,148,075
Trainable params: 9,296,291
Non-trainable params: 23,851,784
_________________________________________________________________
None


In [15]:
logdir = "logs/time_distributed/{0}_{1}/classifier".format(str(dataset), datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

model_path = "models/time_distributed/{0}_{1}/classifier/savefile".format(str(dataset), datetime.now().strftime("%Y%m%d-%H%M%S"))
model_save_callback = keras.callbacks.ModelCheckpoint(filepath=model_path, save_weights_only=True, verbose=1)

In [16]:
class ShuffleCallback(keras.callbacks.Callback):
    def __init__(self, generator):
        self._generator = generator
        
    def on_epoch_end(self, epoch, logs=None):
        self._generator.shuffle()
    
train_shuffle_callback = ShuffleCallback(train_gen)
val_shuffle_callback = ShuffleCallback(val_gen)

In [17]:
model.fit(train_gen, validation_data=val_gen, callbacks=[tensorboard_callback, model_save_callback, train_shuffle_callback, val_shuffle_callback], epochs=20)

Epoch 1/20
 293/1850 [===>..........................] - ETA: 3:17 - loss: 4.4687 - accuracy: 0.0903

KeyboardInterrupt: 

In [ ]:
model.evaluate(test_gen)

30/30 [==============================] - 10s 305ms/step - loss: 7.1514 - accuracy: 0.0083


[7.151444435119629, 0.008333333767950535]